# Project: Wrangling and Analyze Data

## Data Gathering
>In the **Data Gathering** section, the data needed for the project is gathered and made available for assessment, cleaning and analysis.

The methods required to gather each of the three data are different.
1. **WeRateDogs Twitter Archive Data:** This is manually downloaded and uploaded to Jupyter Notebook. File name is `twitter-archive-enhanced.csv`.

2. **Tweet Image Prediction:** Though this is a `tsv` file, it is hosted on Udacity's servers and is to be programmatically downloaded. The Requests library is used to download the file `ìmage_predictions.tsv` from this URL [here](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv).

3. **Additional data from Tweepy library via the Twitter API (tweet_json.txt):** Additional data is queried from the twitter API to add more details and analysis to the report. `Retweet Count` and `Favorite Count` by Twitter IDs will be queried.

In [ ]:
# Import needed packages

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import tweepy
import json
from functools import reduce
import re

### 1. WeRateDogs Twitter Archive Data

In [ ]:
# load the twitter-archive-enhanced.csv file

twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
# Visually assess the data

twitter_archive.head(3)

In [ ]:
#Programmatically assess the data

twitter_archive.info()

In [ ]:
twitter_archive.describe()

### 2. Tweet Image Prediction

In [ ]:
# Download tsv file programmatically

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open('image-predictions.tsv', mode = 'wb') as f:
    f.write(response.content)

# Load the file into a pandas DataFrame

image_prediction_file = pd.read_csv('image-predictions.tsv', sep='\t')


In [ ]:
# Visually assess the image_prediction_file file

image_prediction_file.head()

### 3. Additional data from Tweepy library via the Twitter API (tweet_json.txt)

In [ ]:
# import tweepy


# consumer_key = ''
# consumer_secret = ''
# access_token = ''
# access_secret = ''

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)

# api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# fail_message = []
# with open('tweet_json.txt', 'a', encoding='utf8') as f:
#     for tweet_id in twitter_archive['tweet_id']:
#         try:
#             tweet = api.get_status(tweet_id, tweet_mode='extended')
#             tweet_json = json.dumps(tweet._json)
#             f.write(tweet_json + '\n')
#         except:
#             print(f"Retrieving tweet with ID: {tweet_id} failed")
#             fail_message.append(tweet_id)

In [ ]:
df = []
with open('tweet_json.txt', mode='r') as f:
    #f = file.readlines()
    for each_line in f:
        tweet = json.loads(each_line)
        tweet_id = tweet['id']
        retweet_count = tweet['retweet_count']
        favorite_count = tweet['favorite_count']
        df.append({'tweet_id' : tweet_id,
            'retweet_count' : retweet_count,
                  'favorite_count' : favorite_count})
        
tweet_json_data = pd.DataFrame(df, columns = ['tweet_id', 'retweet_count', 'favorite_count'
                                  ])

In [ ]:
tweet_json_data.head()

## Assessing Data

> In detecting quality and tidiness issues in the three datasets, the data can be assessed both visually and programmatically.

1. **Visual Assessment** comes to play when Data is 'previewed' within the interface of the Jupyter notebook. We can get a quick view of what the data looks like, but could be cumbersome having a view of the whole data, especially for large datasets.
2. **Programmatical Assessment** is when pandas functions and methods are used to assess the data and have a grasp of what the data is.

For the purpose of this analysis, **eight (8) quality issues and two (2) tidiness issues** will be identified programmatically and visually from the data.
It is also important to note that only original ratings that have image will be analysed. Retweets will be excluded from our analysis.

#### Assessing dataset 1 `twitter_archive` to detect quality and/or tidiness issues

In [ ]:
twitter_archive.head(3)

In [ ]:
twitter_archive['source'][0]

In [ ]:
twitter_archive['name'].unique()

In [ ]:
twitter_archive.info()

In [ ]:
twitter_archive.columns.tolist()

In [ ]:
twitter_archive.describe()

#### Assessing dataset 2 `image_prediction_file` to detect quality and/or tidiness issues

In [ ]:
image_prediction_file.head()

In [ ]:
image_prediction_file.info()

In [ ]:
image_prediction_file.describe()

#### Assessing dataset 3 `tweet_json` to detect quality and/or tidiness issues

In [ ]:
tweet_json_data.head()

In [ ]:
tweet_json_data.info()

In [ ]:
tweet_json_data.describe()

### Quality issues
1. Since only original ratings are needed, the replies (`in_reply_to_status_id`, count = 78) and retweets (`retweeted_status_id`, count = 181) in `twitter_archive` is not necessary for the analysis.


2. `image_prediction_file` has a count of 2075 while `twitter_archive` has 2356 rows. Tweets that have no image and even retweets/replies must have been included in the `twitter_archive` data.


3. Since there's no need for retweets and replies, some columns are extraneous on the `twitter_archive` data.
The columns that needs to be removed are **`in_reply_to_status_id`**, **`in_reply_to_user_id`**, **`retweeted_status_id`**, **`retweeted_status_user_id`** and **`retweeted_status_timestamp`**.



4. The **`timestamp`** column on the `twitter_archive` has a consistent tailing '+0000' which is unnecessary.


5. The **`source`** column in the `twitter_archive` dataset still has unneeded html elements which make the column not easy to read and use.


6. The timestamp field has the wrong datatype. This is to be converted to datetime.


7. Inconsistent case in the **`p1`**, **`p2`** and **`p3`** columns as some are lower case while some are sentence case.


8. Some columns do not have good descriptive names.

### Tidiness issues
1. The analysis will be cleaner and easier if the three datasets (`twitter_archive`, `image_prediction_file`, `tweet_json_data`) were merged into a single DataFrame.

2. The **`doggo`**, **`floofer`**, **`pupper`** and **`puppo`** columns on the `twitter_archive` dataset could actually be Melt into a single column and called say, 'dog_type'.

## Cleaning Data

> The data quality and tidiness issues have been identified. Now is time to do the actual cleaning and aligning of the data.
In this section, **all** quality and tidiness issues will be treated.

#### Making copies of original datasets

The cleaning process will involve making a lot of changes to the original datasets. It is appropriate to make copies of the original pieces of data so that work is done only on these copies.

**Note:** _The cleaning process would not sequentially follow the order in which issues are outlined above, but possibly in the order of how a step takes precedence over the next._

In [ ]:
# First make copies of the original datasets

twitter_archive_clean = twitter_archive.copy()
image_prediction_file_clean = image_prediction_file.copy()
tweet_json_data_clean = tweet_json_data.copy()


### Issue #1: 

The analysis will be cleaner and easier if the three datasets (`twitter_archive`, `image_prediction_file`, `tweet_json_data`) were merged into a single DataFrame.

#### Define:

It is tidy to organise the individual datasets into just one. Working on one dataset instead of three ensures we do not make repeatitive actions on each of the datasets, and morever, the data we are working on is really just one data derived from different sources.
These resources; [Real Python](https://realpython.com/pandas-merge-join-and-concat/), [Documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging) and [Stackoverflow](https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes) facilitated my understanding of the pandas merging.

#### Code

In [ ]:
# All three datasets have a similar column (tweet_id) that can be used as primary key to merge
# The three datasets are combined at same time using 'reduce' and 'merge'
combined_data = [twitter_archive_clean, image_prediction_file_clean, tweet_json_data_clean]

merged_df = reduce(lambda  left,right: pd.merge(left,right,on=['tweet_id'], how='outer'
                                            ), combined_data)

In [ ]:
merged_df.head(3)

#### Test

In [ ]:
# Confirm all three datasets are merged into one

merged_df.info()

### Issue #2: 

Data contains reply and retweets which are not needed in the analysis.

#### Define:

Since only original ratings are needed, the replies (`in_reply_to_status_id`, count = 78) and retweets (`retweeted_status_id`, count = 181) in the `merged_df` is not necessary for the analysis.<br><br>Solving this problem, unnecessary items will be removed from the data.

#### Code

In [ ]:
# Removing rows that are actually replies. 
# Rows that have the 'in_reply_to_status_id' populated are replies.
# The method used is retaining only rows where the 'in_reply_to_status_id' is null

merged_df = merged_df[merged_df.in_reply_to_status_id.isnull()]

In [ ]:
# Removing rows that are actually retweets.
# Rows that have the 'retweeted_status_id' populated are retweets.
# The method used is retaining only rows where the 'retweeted_status_id' is null

merged_df = merged_df[merged_df.retweeted_status_id.isnull()]

#### Test

In [ ]:
# Confirm the retweets and replies are no more in the dataset
merged_df.info()

### Issue #3:

Removing tweets that have no image in the `image_prediction_file` data

#### Define

We can see from `merged_df` that the **`jpg_url`** field which belonged to the `image_prediction_file` has 1971 rows while the other records that belonged to `twiter_archive` and `twee_json` has a count of 2097. This implies Tweets that have no image and even retweets/replies must have been included in the `twitter_archive` data.

In solving this issue, only rows that have images will be retained so that those that do not have images are discarded

#### Code

In [ ]:
# Removing rows that do not have images
# This is done by retaining rows with image

merged_df = merged_df[merged_df.jpg_url.notnull()]

#### Test

In [ ]:
# Confirm that records without image are no more in the dataset
merged_df.info()

### Issue #4:

Since there's no need for retweets and replies, some columns are extraneous on the `merged_df` data.
The columns that needs to be removed are **`in_reply_to_status_id`**, **`in_reply_to_user_id`**, **`retweeted_status_id`**, **`retweeted_status_user_id`** and **`retweeted_status_timestamp`**.

#### Code

In [ ]:
# Dropping unneeded columns

merged_df = merged_df.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id',
                            'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1)

#### Test

In [ ]:
# Confirm if the dropped columns are no more in the dataset

merged_df.info()

### Issue #5:

The **`timestamp`** column has a consistent tailing '+0000' which is unnecessary.

#### Code

In [ ]:
# Remove the trailing '+0000' using split

merged_df.timestamp = merged_df.timestamp.str.strip('+0000')

#### Test

In [ ]:
# Confirm if the trailing '+0000' has been removed

merged_df['timestamp'].head()

### Issue #6:

The timestamp field has the wrong datatype.
This is to be converted to datetime.

#### Code

In [ ]:
# Convert the timestamp datatype to datetime

merged_df.timestamp = pd.to_datetime(merged_df.timestamp)

#### Test

In [ ]:
# Confirm if the datatype has been converted to datetime

merged_df.info()

### Issue #7:

The **`source`** column still has unneeded html elements which make the column not easy to read and use.

This will be cleaned-up using regex function

#### Code

In [ ]:
# Investigate the column to confirm how best to clean it

merged_df['source'][0]

In [ ]:
# Remove the trailing '+0000' using split
# Convert to datetime

merged_df['source'] = merged_df['source'].apply(lambda _: re.findall(r'>(.*)<', _)[0])

#### Test

In [ ]:
# Confirm if the html elements have been removed and we only have the source left
# This is done by checking unique values in the 'source column'

merged_df['source'].unique()

### Issue #8:

The **`doggo`**, **`floofer`**, **`pupper`** and **`puppo`** columns on the `twitter_archive` dataset could actually be Melt into a single column and called say, 'dog_type'.

#### Code

In [ ]:
# There are some dogs that doesnt have a dog type

merged_df[['doggo', 'floofer', 'pupper', 'puppo']].head()

In [ ]:
# There are dogs that have 'None' as values for all the dog types (doggo, floofer, pupper and puppo)
# Create a function to add dogs without a dog type to a single column for identification purpose

def no_dog_type(dog):
    if dog['doggo'] == 'None' and dog['floofer'] == 'None' and dog['pupper'] == 'None' and dog['puppo'] == 'None':
        value = 'none_dog_type'
    else:
        value ='None'
    return value

merged_df['unknown_dog_type'] = merged_df.apply(no_dog_type, axis=1)

In [ ]:
merged_df[['name', 'doggo', 'floofer', 'pupper', 'puppo', 'unknown_dog_type']].head(2)

In [ ]:
merged_df.columns

In [ ]:
# Now the 'doggo', 'floofer', 'pupper' and 'puppo' columns are Melt into a single column, 'dog_type'
merged_df = pd.melt(merged_df, id_vars =['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog',
       'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'retweet_count',
       'favorite_count'],
                     value_vars = ['doggo', 'floofer', 'pupper', 'puppo', 'unknown_dog_type'],
                     var_name = 'dog_type', 
                    value_name = 'value')

In [ ]:
# Remove all duplicated rows and drop the 'value' column

merged_df = merged_df[merged_df['value']!= 'None']
merged_df = merged_df.drop('value', axis=1)

#### Test

In [ ]:
# Confirm if all steps applied worked

merged_df.columns

In [ ]:
merged_df.head(2)

In [ ]:
merged_df.info()

### Issue #9:

Inconsistent case in the **`p1`**, **`p2`** and **`p3`** columns as some are lower case while some are sentence case.

This is a quality issue that will be cleaned by making all the values in all three columns to be lower case.

In [ ]:
# Preview the data to see the varying cases in the 'p1', 'p2' and 'p3' columns.

merged_df[['p1', 'p2', 'p3']].iloc[0:10]

#### Code

In [ ]:
# Clean-up the issue by applying lowercase to all values in the 'p1', 'p2' and 'p3' columns

merged_df[['p1', 'p2', 'p3']] = merged_df[['p1', 'p2', 'p3']].apply(lambda x: x.str.lower())

#### Test

In [ ]:
#Confirm this has been cleaned up

merged_df[['p1', 'p2', 'p3']].iloc[0:10]

### Issue #10:

Rename columns appropriately

In [ ]:
# Preview the data to investigate the columns and determine which ones need
# to be renamed

merged_df.info()

In [ ]:
merged_df = merged_df.rename(columns = {'timestamp':'tweet_date',
                           'source': 'tweet_source',
                           'text': 'tweet_text',
                           'expanded_urls': 'tweet_url',
                           'name': 'name_of_dog',
                           'jpg_url': 'image_link',
                           'breed': 'dog_breed'})

#### Code

In [ ]:
# Rename the 'source', 'text', 'expanded_urls', 'name', 'jpg_url' and breed column names
# to more descriptive names
merged_df = merged_df.rename(columns = {'timestamp':'tweet_date',
                           'source': 'tweet_source',
                           'text': 'tweet_text',
                           'expanded_urls': 'tweet_url',
                           'name': 'name_of_dog',
                           'jpg_url': 'image_link',
                           'breed': 'dog_breed'})

#### Test

In [ ]:
#Confirm this has been cleaned up

merged_df.info()

## Storing Data

The `merged_df` DataFrame that has been worked on so far is stored to a CSV file named "twitter_archive_master.csv".

In [ ]:
merged_df.to_csv("twitter_archive_master.csv", index=False)

## Analyzing and Visualizing Data

In this section, the wrangled data is analyzed and visualized by producing insights and visualizations.

In [ ]:
# Read the file containing the wrangled data

df = pd.read_csv("twitter_archive_master.csv")

In [ ]:
df.head(3)

In [ ]:
df.info()

### Insights:
1. Which dogs had the highest ratings/scores

2. Which dogs had the highest favourites (likes)

3. Which dog_type had the highest favourites (likes)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#### 1. Which dogs had the highest ratings/scores

In [ ]:
df.groupby('name_of_dog')['rating_numerator'].mean().sort_values(ascending=False).nlargest(10)

> Atticus is the winner here! Atticus had the highest average score

In [ ]:
# Plot bar chart of the highest rated 10 dogs

fig = plt.figure(figsize=(10,5))
df.groupby('name_of_dog')['rating_numerator'].mean().sort_values(ascending=False).nlargest(10).plot(kind='bar')
plt.title("Top 10 Highest Rated Dogs",fontsize=15)
plt.ylabel("Dog Name")
plt.xlabel("Average Ratings"); 

#### 2. Which dogs had the highest favourites (likes)

In [ ]:
df.groupby('name_of_dog')['favorite_count'].mean().sort_values(ascending=False).nlargest(10)

> Stephan is the winner on this one, and is closely followed by Jamesy.<br><br>Now, we all like Stephan 🥰

In [ ]:
# Plot bar chart of the highest liked 10 dogs

fig = plt.figure(figsize=(10,5))
df.groupby('name_of_dog')['favorite_count'].mean().sort_values(ascending=False).nlargest(10).plot(kind='bar')
plt.title("Top 10 Highest Liked Dogs",fontsize=15)
plt.ylabel("Dog Name")
plt.xlabel("Average Favourite Count"); 


#### 3. Which dog_type had the highest favourites (likes)

In [ ]:
# Exclude where dog_type = 'unknown_dog_type' to make the analysis cleaner

df = df[df['dog_type'] != 'unknown_dog_type']

In [ ]:
df.groupby('dog_type')['favorite_count'].mean().sort_values(ascending=False)

> Puppo wins the race! I'll go get a puppo...<br><br>That's a cute dog🥰

In [ ]:
# Plot a pie chart to show how dog types faired against each other

fig = plt.figure(figsize=(8,4))
df.groupby('dog_type')['favorite_count'].mean().sort_values(ascending=False).plot(kind='pie')
plt.title("Top Liked Dog Types",fontsize=15)
plt.ylabel("Dog Type")
plt.xlabel("Average Favourite Count");